<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks<a class="jp-InternalAnchorLink" href="#unSkript-Runbooks" target="_self">&para;</a></h1>
<div class="alert alert-block alert-success">
<h3 id="Objective"><strong>Objective</strong><a class="jp-InternalAnchorLink" href="#Objective" target="_self">&para;</a></h3>
<strong>This runbook demonstrates How to Remediate unencrypted S3 buckets.</strong></div>
</center><center>
<h2 id="Remediate-unencrypted-S3-buckets">Remediate unencrypted S3 buckets<a class="jp-InternalAnchorLink" href="#Remediate-unencrypted-S3-buckets" target="_self">&para;</a></h2>
</center>
<h1 id="Steps-Overview">Steps Overview<a class="jp-InternalAnchorLink" href="#Steps-Overview" target="_self">&para;</a></h1>
<ol>
<li>Filter all the S3 buckets which are unencrypted.</li>
<li>Apply encryption on unencrypted S3 buckets.</li>
</ol>

In [5]:
if bucket_name and not region:
    raise SystemExit("Provide region for the S3 Bucket!")
if region == None:
    region = ""

<h3 id="Filter-AWS-Unencrypted-S3-Buckets">Filter AWS Unencrypted S3 Buckets<a class="jp-InternalAnchorLink" href="#Filter-Unattached-EBS-Volumes" target="_self">&para;</a></h3>
<p>Here we will use unSkript <strong>Filter Unencrypted S3 Buckets</strong> action. This action filters all the S3 buckets from the given region and returns a list of those S3 buckets without encryption. It will execute if the bucket_name parameter is not given.</p>
<blockquote>
<p>Input parameters: <code>region</code></p>
</blockquote>
<blockquote>
<p>Output variable: <code>unencrypted_buckets</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, Tuple
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
from botocore.exceptions import ClientError
import pprint


from beartype import beartype
@beartype
def aws_filter_unencrypted_s3_buckets_printer(output):
    if output is None:
        return

    pprint.pprint(output)


@beartype
def aws_filter_unencrypted_s3_buckets(handle, region: str = "") -> Tuple:
    """aws_filter_unencrypted_s3_buckets List of unencrypted S3 bucket name .

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: string
        :param region: Filter S3 buckets.

        :rtype: Tuple with status result and list of unencrypted S3 bucket name.
    """
    result = []
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)
    for reg in all_regions:
        try:
            s3Client = handle.client('s3', region_name=reg)
            response = s3Client.list_buckets()
            # List unencrypted S3 buckets
            for bucket in response['Buckets']:
                try:
                    response = s3Client.get_bucket_encryption(Bucket=bucket['Name'])
                    encRules = response['ServerSideEncryptionConfiguration']['Rules']
                except ClientError as e:
                    bucket_dict = {}
                    bucket_dict["region"] = reg
                    bucket_dict["bucket"] = bucket['Name']
                    result.append(bucket_dict)
        except Exception as error:
            pass

    if len(result) != 0:
        return (False, result)
    else:
        return (True, None)



task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not bucket_names",
    "condition_result": true
    }''')
task.configure(outputName="unencrypted_buckets")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_unencrypted_s3_buckets, lego_printer=aws_filter_unencrypted_s3_buckets_printer, hdl=hdl, args=args)

<h3 id="Modify-Unencrypted-S3-Buckets-Output">Modify Unencrypted S3 Buckets Output</h3>
<p>In this action, we modify the output from step 1 and return a list of dictionary items for the Unencrypted S3 Buckets</p>
<blockquote>
<p>Output variable: ebs_list</p>
</blockquote>

In [ ]:
bucket_list = []

try:
    if unencrypted_buckets[0] == False:
        for bucket in unencrypted_buckets[1]:
            bucket_list.append(bucket)
except Exception as e:
    if bucket_names:
        for i in bucket_names:
            data_dict = {}
            data_dict["region"] = region
            data_dict["bucket"] = i
            bucket_list.append(data_dict)
    else:
        raise Exception(e)

<h3 id="Apply-AWS-Default-Encryption-for-S3-Bucket">Apply AWS Default Encryption for S3 Bucket</h3>
<p>Here we will use the unSkript <strong>Apply AWS Default Encryption for the S3 Buckets</strong> action. In this action, we will apply the default encryption configuration to the unencrypted S3 buckets by passing the list of unencrypted S3 buckets from step 1.</p>
<blockquote>
<p>Input parameters: <code>name</code>, <code>region</code></p>
</blockquote>
<blockquote>
<p>Output variable: <code>apply_output</code></p>
</blockquote>

In [2]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Dict
import json
import pprint


from beartype import beartype
@beartype
def aws_put_bucket_encryption_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_put_bucket_encryption(handle, name: str, region: str) -> Dict:
    """aws_put_bucket_encryption Puts default encryption configuration for bucket.

          :type name: string
          :param name: NAme of the S3 bucket.

          :type region: string
          :param region: location of the bucket

          :rtype: Dict with the response info.
      """
    s3Client = handle.client('s3',
                             region_name=region)

    # Setup default encryption configuration 
    response = s3Client.put_bucket_encryption(
        Bucket=name,
        ServerSideEncryptionConfiguration={
            "Rules": [
                {"ApplyServerSideEncryptionByDefault": {"SSEAlgorithm": "AES256"}}
            ]},
        )
    return response


task = Task(Workflow())
task.configure(printOutput=True)
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "iter.get(\\"region\\")",
    "name": "iter.get(\\"bucket\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "bucket_list",
    "iter_parameter": ["name","region"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(bucket_list) > 0",
    "condition_result": true
    }''')
task.configure(outputName="apply_output")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_put_bucket_encryption, lego_printer=aws_put_bucket_encryption_printer, hdl=hdl, args=args)

### Conclusion
<p>In this Runbook, we demonstrated the use of unSkript's AWS legos to filter all unencrypted S3 buckets and apply default encryption configuration to the buckets. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">https://us.app.unskript.io</a></p>